In [1]:
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_boston
import numpy as np
import collections
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
# data preprocession

boston_data = load_boston()
X = boston_data.data
y = boston_data.target.reshape(X.shape[0],1)
x = [[1] for i in range(X.shape[0])]
X = np.concatenate((x, X), axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X.shape, y.shape
X_train.shape, y_train.shape
X_test.shape, y_test.shape

((506, 14), (506, 1))

((404, 14), (404, 1))

((102, 14), (102, 1))

In [22]:
def calLinearWeight(X, y):
    return np.linalg.pinv(X.T.dot(X)).dot(X.T).dot(y)
    
def calRidgeWeight(X, y, lamb):
    return np.linalg.pinv(lamb * np.eye(X.shape[1]) + X.T.dot(X)/ X.shape[0]).dot(X.T).dot(y) / X.shape[0]

def calRSS(W, X, y):
    _y = X.dot(W)
    return np.sum((y-_y) ** 2)

def calTSS(y):
    return np.sum((y-np.mean(y)) ** 2)

def calR2(W, X, y):
    return 1 - calRSS(W, X, y)/calTSS(y)

def findLambda(X, y, candidates):
    '''
    Use cross validation to find the best lambda
    '''
    N = X.shape[0]
    k = 10
    v_size = N // k
    RSS = collections.defaultdict()
    for l in candidates:
        all_RSS = 0
        for i in range(0, N, v_size):
            Xt = np.append(X[:i], X[i+v_size:], axis=0)
            Xv = X[i:i+v_size]
            yt = np.append(y[:i], y[i+v_size:], axis=0)
            yv = y[i:i+v_size]
            tmp_W = calRidgeWeight(Xt, yt, l)
            all_RSS += calRSS(tmp_W, Xv, yv)
        RSS[l] = all_RSS / k
        print('lambda', l, 'RSS:', all_RSS/k)
    return min(RSS.items(), key=lambda x:x[1])[0]

def polyTrans2(X):
    tmp = X.T
    trans_X = tmp[:1]
    N = X.shape[0]

    for i in range(1, X.shape[1]):
        trans_X = np.append(trans_X, tmp[i].reshape(1, N), axis=0)
        for j in range(i, X.shape[1]):
            x = np.multiply(tmp[i], tmp[j])
            trans_X = np.append(trans_X, x.reshape(1, N), axis=0)
    return trans_X.T

def report(W, Xtrain, Xtest, model):
    print(model,':')
    print('Train RSS =', calRSS(W, Xtrain, y_train))
    print('Train TSS =', calTSS(y_train))
    print('Train R2 =', calR2(W, Xtrain, y_train))

    print('Test RSS =', calRSS(W, Xtest, y_test))
    print('Test TSS =', calTSS(y_test))
    print('Test R2 =', calR2(W, Xtest, y_test))

In [4]:
# print RSS, TSS, R^2 of the training and test data using Linear Regression Model

lr_W = calLinearWeight(X_train, y_train)
report(lr_W, X_train, X_test, 'Linear regression model')

Linear regression model :
Train RSS = 8743.13075230343
Train TSS = 35096.85514851485
Train R2 = 0.7508856358979673
Test RSS = 2477.6941864655173
Test TSS = 7480.045882352942
Test R2 = 0.6687594935331964


In [5]:
best_lambda = findLambda(X_train, y_train, [0.001, 0.0005, 0.0002, 0.000165, 0.000164, 0.00016, 0.0001])
print('Best lambda is:', best_lambda)

lambda 0.001 RSS: 985.5303571598619
lambda 0.0005 RSS: 981.0336705807644
lambda 0.0002 RSS: 979.4465881763223
lambda 0.000165 RSS: 979.4201043290674
lambda 0.000164 RSS: 979.4200860362125
lambda 0.00016 RSS: 979.4204455452252
lambda 0.0001 RSS: 979.5166725953457
Best lambda is: 0.000164


In [6]:
# print RSS, TSS, R^2 of the training and test data using Ridge Regression Model with lambda=0.000164

rr_W = calRidgeWeight(X_train, y_train, best_lambda)
report(rr_W, X_train, X_test, 'Ridge regression model')

Ridge regression model :
Train RSS = 8750.616788114796
Train TSS = 35096.85514851485
Train R2 = 0.7506723394137185
Test RSS = 2507.544019049138
Test TSS = 7480.045882352942
Test R2 = 0.6647688986821617


In [20]:
trans_X = polyTrans2(X)
trans_X_train, trans_X_test, y_train, y_test = train_test_split(trans_X, y, test_size=0.2, random_state=42)
trans_X_train.shape

(404, 105)

In [23]:
lrt_W = calLinearWeight(trans_X_train, y_train)
report(lrt_W, trans_X_train, trans_X_test, 'Linear regression with polynomial transformation model')

Linear regression with polynomial transformation model :
Train RSS = 2214.1775807592558
Train TSS = 35096.85514851485
Train R2 = 0.9369123651851482
Test RSS = 1339.516034046506
Test TSS = 7480.045882352942
Test R2 = 0.8209214147727735


In [55]:
best_lambda2 = findLambda(trans_X_train, y_train, [0.001, 0.0001, 0.00009, 0.000088, 0.000085])
print('Best lambda is:', best_lambda2)

lambda 0.001 RSS: 795.961593978952
lambda 0.0001 RSS: 779.2118934788183
lambda 9e-05 RSS: 775.2466889295263
lambda 8.8e-05 RSS: 774.1482519992259
lambda 8.5e-05 RSS: 797.2605518672135
Best lambda is: 8.8e-05


In [25]:
# print RSS, TSS, R^2 of the training and test data using Ridge Regression with polynomial transformation Model with lambda=8.8e-05

rrt_W = calRidgeWeight(trans_X_train, y_train, best_lambda2)
report(rrt_W, trans_X_train, trans_X_test, 'Ridge regression with polynomial transformation model')

Ridge regression with polynomial transformation model :
Train RSS = 2129.584945611813
Train TSS = 35096.85514851485
Train R2 = 0.9393226277226172
Test RSS = 1374.5312924632642
Test TSS = 7480.045882352942
Test R2 = 0.8162402592066871


In [41]:
lrt_W.shape
lrt_W.T

(105, 1)

array([[ 1.45725898e-01, -1.11337704e+01,  1.88300630e-04,
         1.04067485e-01,  2.53487878e-01,  2.92143509e+00,
        -1.28121053e+00,  2.12249261e-01, -1.92454512e-03,
        -9.09796460e-02, -1.53193299e-01, -1.14701084e-03,
         5.06520118e-01, -4.61670707e-04,  3.17390216e-02,
         6.26069011e-01,  1.89131906e-04, -2.80546372e-03,
        -4.11886075e-02, -1.17371518e+00, -1.22669263e-02,
         1.10631412e-03, -2.38476900e-02, -1.54558650e-02,
         6.85394644e-04, -1.37574520e-03,  2.07990678e-05,
        -1.00694778e-02, -2.05822309e+00,  3.39362582e-02,
        -3.46936280e-01, -7.00350252e-02,  1.47254624e-01,
         4.54533225e-03,  5.07776999e-02, -8.09100557e-02,
         6.93626811e-04, -5.43021305e-02,  3.17955809e-03,
        -2.14970989e-02, -3.60124328e-01, -3.60124326e-01,
        -2.33917279e+00, -4.13007417e+00,  4.92914876e-02,
         3.89196377e-01, -8.36165906e-01,  9.44887406e-03,
         1.23143456e+00,  3.43437248e-02, -5.42832973e-0

In [42]:
# Predict a new house price.

X_pre = [1, 5, 0.5, 2, 0, 4, 8, 4, 6, 2, 2, 2, 4, 5.5]
X_pre = np.array(X_pre)
X_pre = X_pre.reshape(1, X_pre.shape[0])
trans_X_pre = polyTrans2(X_pre)
y_pre = trans_X_pre.dot(lrt_W)
y_pre

array([[439.95654362]])

In [53]:
trans_X_pre.shape
X_pre.shape
X_pre.dot(lr_W)
X_pre.dot(rr_W)
trans_X_pre.dot(lrt_W)
trans_X_pre.dot(rrt_W)

(1, 105)

(1, 14)

array([[-16.31002566]])

array([[-10.44873713]])

array([[439.95654362]])

array([[389.11703453]])